<a href="https://colab.research.google.com/github/apoorvkh/notebooks/blob/main/Face%20Recognition%20(ArcFace).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation

In [ ]:
%%bash
git clone https://github.com/peteryuX/arcface-tf2.git arcface
pip install -r arcface/requirements.txt

In [2]:
import requests
import zipfile

def download(file_id, dst_path):
    URL = "https://docs.google.com/uc?export=download"
    session = requests.Session()
    response = session.get(URL, params = { 'id' : file_id }, stream = True)

    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            params = { 'id' : file_id, 'confirm' : value }
            response = session.get(URL, params = params, stream = True)
            break

    with open(dst_path, "wb") as f:
        for chunk in response.iter_content(32768):
            if chunk:
                f.write(chunk)

download('1HasWQb86s4xSYy36YbmhRELg9LBmvhvt', 'arc_res50.zip')
with zipfile.ZipFile('arc_res50.zip', 'r') as zip_ref:
    zip_ref.extractall('checkpoints')
!rm arc_res50.zip

# Loading model

In [3]:
import tensorflow as tf
import numpy as np
import cv2
from arcface.modules.models import ArcFaceModel
from arcface.modules.utils import set_memory_growth, l2_norm

set_memory_growth()

model = ArcFaceModel(size=112, backbone_type='ResNet50', training=False)
model.load_weights('./checkpoints/arc_res50/e_8_b_40000.ckpt')

def preprocess_img(img):
    img = cv2.resize(img, (112, 112))
    img = img.astype(np.float32) / 255.
    if len(img.shape) == 3:
        img = np.expand_dims(img, 0)
    return img

# Input: BGR image as np array
def compute_embeddings(img):
    img = preprocess_img(img)
    emb = l2_norm(model(img))
    return emb

94773248/94765736 [==============================] - 7s 0us/step


# Inference

In [4]:
img = cv2.imread('arcface/data/BruceLee.jpg')
embedding = compute_embeddings(img)

print(embedding.shape)

(1, 512)
